In [1]:
import stumpy
import numpy as np
import random
import pickle

from tqdm.auto import tqdm
from multiprocessing import Pool
from itertools import product

import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
def distance_to_shapelet(data, shapelets):    
    # processed output data
    data_out = np.zeros((len(data),len(shapelets)))
    
    # loop over each sample in the dataset
    for i,sample in enumerate(tqdm(data)):
        shapelet_score = np.empty(len(shapelets))
        # for each shapelet, calculate distance and assign a score
        for j,shapelet in enumerate(shapelets):
            try:
                dist = stumpy.mass(shapelet, sample)
            except ValueError:
                dist = stumpy.mass(sample, shapelet)
            shapelet_score[j] = dist.min()
        data_out[i] = shapelet_score
    
    return data_out

def process_traces(shapelets, name):
    X, y = [], []

    # iterate over dictionary and re-format into X and y
    for trace_id, trace_vals in traces.items():
        for trace in trace_vals:
            X.append(trace)
            y.append(trace_id)
    
    print("Processing" + name + "... " + "(" + str(len(X)) + " traces)")
    
    # convert traces into float64 data type
    X = [np.asarray(trace).astype('float64') for trace in X]
    # clear empty trace values in data
    X = [trace[~np.isnan(trace)] for trace in X]    
    # compute distance between input trace and shapelet arrays
    # return as new X
    X = distance_to_shapelet(X, shapelets)
    
    return X, y

In [3]:
# note: python multiprocessing is really annoying to work with
# function needs to be in a separate .py file which is imported
# and function can only have 1 argument
# list input which is immediately used for what would be the arguments
def evaluate_parameters(namestring):
    
    print(namestring)
    
    files = {
        'shapelets': folder_shapelets + namestring,
        'X': folder_X + namestring,
        'y': folder_y + namestring
    }
    try:
        with open(files['shapelets'], 'rb') as f:
            shapelets = pickle.load(f)
    except FileNotFoundError:
        print("Shapelet File Missing:" + files['shapelets'] + ", skipping...")
        return
    
    shapelets = [shapelet.astype('float64') for shapelet in shapelets]
    
    X, y = process_traces(shapelets, files['shapelets'])
    
    with open(files['X'], 'wb') as f:
        pickle.dump(X, f)
        
    with open(files['y'], 'wb') as f:
        pickle.dump(y, f)

In [4]:
def classifier_performance(X, y, perclass, cv=False):
    
    clf = RandomForestClassifier()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    if cv:
        return np.mean(cross_val_score(clf, X, y, cv=5))
    if perclass:
        matrix = metrics.confusion_matrix(y_test, y_pred)
        scores = matrix.diagonal()/matrix.sum(axis=1)
    else:
        scores = metrics.accuracy_score(y_test, y_pred)
    
    return scores

In [5]:
def make_namestring_list(namestring_dict):
    
    name_components = []
    for cat in namestring_dict:
        values = namestring_dict[cat]
        name_components.append([str(cat) + "=" + str(value) for value in values])
    
    namestring_list = [''.join(item) for item in product(*name_components)]
    
    return namestring_list

In [6]:
def merge_x(namestring_list, verbose=True):
    
    X = ()

    for filename in namestring_list:
        if verbose: print(filename)
        with open(folder_X + filename, 'rb') as f:
            Xi = pickle.load(f)
        X = X + (Xi,) 

    X = np.concatenate(X, axis=1)

    with open(folder_y + namestring_list[0], 'rb') as f:
        y = pickle.load(f)
    y = np.array(y) 
    
    if verbose: print(X.shape);print(y.shape);
    
    return X, y

In [7]:
def batch_classifier(namestring_list, perclass=False, repeat=1):
        
    for name in namestring_list:
        with open(folder_X + name, 'rb') as f:
            X = pickle.load(f)
        with open(folder_y + name, 'rb') as f:
            y = pickle.load(f)
            
        all_scores = []
        for i in range(repeat):
            clf = RandomForestClassifier()
            scores = classifier_performance(X, y, perclass, True)
            all_scores.append(scores)
        
        print(name + ": " + str(all_scores))
        with open(folder_scores + name, 'wb') as f:
            pickle.dump(all_scores, f)

def batch_merged_classifier(listof_namestring_list, perclass=False, repeat=1):
    for namestring_list in listof_namestring_list:
        X, y = merge_x(namestring_list, False)
        
        all_scores = []
        for i in range(repeat):
            clf = RandomForestClassifier()
            scores = classifier_performance(X, y, perclass, True)
            all_scores.append(scores)
        
        outfile = '-'.join(namestring_list) + "_merged"
        print(outfile + ": " + str(all_scores))
        with open(folder_scores + outfile, 'wb') as f:
            pickle.dump(all_scores, f)

In [8]:
# SETUP GLOBAL VARIABLES
# techincally, in jupyter notebook these dont need to be global and everything will run fine
# for the the sake of clarity, I have made them global

global traces
with open("../datasets/ds19.npy", 'rb') as f:
    traces = pickle.load(f)
global folder_scores
folder_scores = "../results/scores/"
global folder_shapelets
folder_shapelets = "../results/shapelets/"
global folder_X
folder_X = "../results/data/X/"
global folder_y
folder_y = "../results/data/y/"

In [9]:
## PART 2

namestring_list = make_namestring_list({'n_clusters':[2,3,4], 'num':[*range(4)]})
print(namestring_list)

if __name__ == '__main__':

    from utils import evaluate_parameters
    
    with Pool(6) as p:
        p.map(evaluate_parameters, namestring_list)

['n_clusters=2num=0', 'n_clusters=2num=1', 'n_clusters=2num=2', 'n_clusters=2num=3', 'n_clusters=3num=0', 'n_clusters=3num=1', 'n_clusters=3num=2', 'n_clusters=3num=3', 'n_clusters=4num=0', 'n_clusters=4num=1', 'n_clusters=4num=2', 'n_clusters=4num=3']
n_clusters=2num=2
Shapelet File Missing:../results/shapelets/n_clusters=2num=2, skipping...
n_clusters=3num=2
Processing../results/shapelets/n_clusters=3num=2... (10000 traces)
n_clusters=2num=0
Processing../results/shapelets/n_clusters=2num=0... (10000 traces)
n_clusters=2num=1
Processing../results/shapelets/n_clusters=2num=1... (10000 traces)
n_clusters=2num=3
Shapelet File Missing:../results/shapelets/n_clusters=2num=3, skipping...
n_clusters=3num=3
Shapelet File Missing:../results/shapelets/n_clusters=3num=3, skipping...
n_clusters=4num=0
Processing../results/shapelets/n_clusters=4num=0... (10000 traces)
n_clusters=3num=1
Processing../results/shapelets/n_clusters=3num=1... (10000 traces)
n_clusters=3num=0
Processing../results/shapele

100%|██████████| 10000/10000 [48:45<00:00,  3.42it/s] 


n_clusters=4num=1
Processing../results/shapelets/n_clusters=4num=1... (10000 traces)


  0%|          | 4/10000 [00:00<27:06,  6.15it/s]/s]

n_clusters=4num=2
Processing../results/shapelets/n_clusters=4num=2... (10000 traces)


100%|█████████▉| 9988/10000 [48:48<00:04,  2.67it/s]

n_clusters=4num=3
Processing../results/shapelets/n_clusters=4num=3... (10000 traces)


100%|██████████| 10000/10000 [48:34<00:00,  3.43it/s]


In [26]:
batch_classifier(make_namestring_list({'num':[15],'size':[0]}))

num=15size=0: [0.857]


In [24]:
for i in range(2,6):
    listof_namestring_list = [make_namestring_list({'num':random.sample(range(36), i), 'size':[0]}) for j in range(1)]
    batch_merged_classifier(listof_namestring_list, repeat=1)

num=26size=0-num=33size=0_merged: [0.8752000000000001]
num=15size=0-num=13size=0-num=28size=0_merged: [0.8764]
num=26size=0-num=33size=0-num=30size=0-num=15size=0_merged: [0.8811]
num=14size=0-num=22size=0-num=6size=0-num=19size=0-num=33size=0_merged: [0.8787]


In [25]:
listof_namestring_list = [
    make_namestring_list({'n_clusters':'2','num':[*range(2)]}),
    make_namestring_list({'n_clusters':'3','num':[*range(3)]}),
    make_namestring_list({'n_clusters':'4','num':[*range(4)]}),
]

print(listof_namestring_list)

batch_merged_classifier(listof_namestring_list, repeat=1)

[['n_clusters=2num=0', 'n_clusters=2num=1'], ['n_clusters=3num=0', 'n_clusters=3num=1', 'n_clusters=3num=2'], ['n_clusters=4num=0', 'n_clusters=4num=1', 'n_clusters=4num=2', 'n_clusters=4num=3']]
n_clusters=2num=0-n_clusters=2num=1_merged: [0.8872]
n_clusters=3num=0-n_clusters=3num=1-n_clusters=3num=2_merged: [0.8913]
n_clusters=4num=0-n_clusters=4num=1-n_clusters=4num=2-n_clusters=4num=3_merged: [0.8943999999999999]


In [10]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK

In [12]:
# Create training, testing, and validation sets
X,y = merge_x(make_namestring_list({'n_clusters':'4','num':[*range(4)]}), False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1000)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=1000)

dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dval_reg = xgb.DMatrix(X_val, y_val, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [11]:
def score(params):
    print("Training with: ", end="")
    print(params)
    
    num_boost_round = params['num_boost_round']
    del params['num_boost_round']
    
    model = xgb.train(
        params=params,
        dtrain=dtrain_reg,
        num_boost_round=num_boost_round,
    )
    
    y_pred = model.predict(dval_reg)
    score = metrics.accuracy_score(y_val, y_pred)
    return {'loss': 1 - score, 'status': STATUS_OK,}
    

In [22]:
# XGBoost Hyper-parameter optimization
search_space = {
    "objective": "multi:softmax",
    "num_class": 100,
    "booster": "gbtree",
    "sampling_method": 'uniform',
    "num_boost_round": hp.randint("num_boost_round", 100) + 100,
    "eta": hp.uniform("eta", 0, 1),
    "gamma": hp.lognormal("gamma", 0, 1),
    "max_depth": hp.randint("max_depth", 9) + 1,
    "min_child_weight": hp.lognormal("min_child_weight", 0, 1),
    "max_delta_step": hp.lognormal("max_delta_step", 0, 1),
    "subsample": hp.uniform("subsample", 0.5, 1),
    "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1),
    "colsample_bylevel": hp.uniform("colsample_bylevel", 0.5, 1),
    "colsample_bynode": hp.uniform("colsample_bynode", 0.5, 1),
    "lambda": hp.lognormal("lambda", 0, 1),
    "alpha": hp.lognormal("alpha", 0, 1),
    "tree_method": hp.choice("tree_method", ['auto', 'exact','approx','hist']),
    "grow_policy": hp.choice("grow_policy", ['depthwise', 'lossguide']),
    "eval_metric": hp.choice("eval_metric", ['rmse','rmsle','mae','mape','mphe','mlogloss','merror','map']) 
}

In [24]:
best = fmin(score, search_space, algo=tpe.suggest, max_evals=150)

Training with:                                                                  
{'alpha': 1.057018515513093, 'booster': 'gbtree', 'colsample_bylevel': 0.7229313974049449, 'colsample_bynode': 0.9952138061032817, 'colsample_bytree': 0.9618359742190566, 'eta': 0.12033595491593219, 'eval_metric': 'map', 'gamma': 0.8137873990214178, 'grow_policy': 'depthwise', 'lambda': 0.4247320116555457, 'max_delta_step': 0.5706701046441234, 'max_depth': 1, 'min_child_weight': 0.05835703978300715, 'num_boost_round': 183, 'num_class': 100, 'objective': 'multi:softmax', 'sampling_method': 'uniform', 'subsample': 0.9212110958357833, 'tree_method': 'exact'}
Training with:                                                                  
{'alpha': 1.1721281204496534, 'booster': 'gbtree', 'colsample_bylevel': 0.9852304293578997, 'colsample_bynode': 0.7514072087814645, 'colsample_bytree': 0.6065920514991698, 'eta': 0.32590042843772526, 'eval_metric': 'mlogloss', 'gamma': 1.5640587426101467, 'grow_policy': 'dept

Training with:                                                                  
{'alpha': 0.5966282629152805, 'booster': 'gbtree', 'colsample_bylevel': 0.8292461960049706, 'colsample_bynode': 0.9031915132207934, 'colsample_bytree': 0.5381092539981225, 'eta': 0.8978281675218416, 'eval_metric': 'mae', 'gamma': 0.6334561301933432, 'grow_policy': 'depthwise', 'lambda': 0.4321939722601179, 'max_delta_step': 0.38747148794946934, 'max_depth': 6, 'min_child_weight': 1.7387779638345804, 'num_boost_round': 112, 'num_class': 100, 'objective': 'multi:softmax', 'sampling_method': 'uniform', 'subsample': 0.6726584369258393, 'tree_method': 'approx'}
Training with:                                                                  
{'alpha': 1.422311133827155, 'booster': 'gbtree', 'colsample_bylevel': 0.9646169658273736, 'colsample_bynode': 0.9077293885888713, 'colsample_bytree': 0.5178453159853826, 'eta': 0.5423246321944946, 'eval_metric': 'mlogloss', 'gamma': 0.11623760881752469, 'grow_policy': 'loss

Training with:                                                                  
{'alpha': 0.7979037966829601, 'booster': 'gbtree', 'colsample_bylevel': 0.508706187001514, 'colsample_bynode': 0.7896624387879073, 'colsample_bytree': 0.8589995871108447, 'eta': 0.25589379079862135, 'eval_metric': 'rmse', 'gamma': 0.5070820574617547, 'grow_policy': 'depthwise', 'lambda': 1.8458780548953353, 'max_delta_step': 1.6855477368469745, 'max_depth': 6, 'min_child_weight': 14.957948152091822, 'num_boost_round': 173, 'num_class': 100, 'objective': 'multi:softmax', 'sampling_method': 'uniform', 'subsample': 0.7474859291294336, 'tree_method': 'approx'}
Training with:                                                                  
{'alpha': 1.2388523092756347, 'booster': 'gbtree', 'colsample_bylevel': 0.6990765459178911, 'colsample_bynode': 0.718759521038416, 'colsample_bytree': 0.8137239848597473, 'eta': 0.4093523721586183, 'eval_metric': 'mape', 'gamma': 0.24983598978045246, 'grow_policy': 'lossguid

Training with:                                                                  
{'alpha': 0.8461543273416059, 'booster': 'gbtree', 'colsample_bylevel': 0.6349636608372631, 'colsample_bynode': 0.8415748674158193, 'colsample_bytree': 0.9210609267077464, 'eta': 0.7118526760855726, 'eval_metric': 'rmsle', 'gamma': 3.381466709430099, 'grow_policy': 'depthwise', 'lambda': 2.9251313962629224, 'max_delta_step': 0.1829623095789454, 'max_depth': 1, 'min_child_weight': 1.317377978539998, 'num_boost_round': 106, 'num_class': 100, 'objective': 'multi:softmax', 'sampling_method': 'uniform', 'subsample': 0.6333969421263012, 'tree_method': 'auto'}
Training with:                                                                  
{'alpha': 0.7232298499092071, 'booster': 'gbtree', 'colsample_bylevel': 0.5574967177902553, 'colsample_bynode': 0.9672252426430346, 'colsample_bytree': 0.9723742757871042, 'eta': 0.8103143665979428, 'eval_metric': 'mphe', 'gamma': 0.35713357565668763, 'grow_policy': 'lossguide'

Training with:                                                                  
{'alpha': 1.5559054675289536, 'booster': 'gbtree', 'colsample_bylevel': 0.6829089914515906, 'colsample_bynode': 0.7200248161858527, 'colsample_bytree': 0.5682764706888983, 'eta': 0.3545492498182623, 'eval_metric': 'map', 'gamma': 1.0199108536513946, 'grow_policy': 'depthwise', 'lambda': 1.9839298613634806, 'max_delta_step': 2.8898747108941247, 'max_depth': 2, 'min_child_weight': 0.9528362668850477, 'num_boost_round': 125, 'num_class': 100, 'objective': 'multi:softmax', 'sampling_method': 'uniform', 'subsample': 0.9917246417055641, 'tree_method': 'approx'}
Training with:                                                                  
{'alpha': 3.1765261143306973, 'booster': 'gbtree', 'colsample_bylevel': 0.5706565801773884, 'colsample_bynode': 0.667241314312337, 'colsample_bytree': 0.7520914636754421, 'eta': 0.9842625911106888, 'eval_metric': 'rmsle', 'gamma': 0.3412538058917156, 'grow_policy': 'lossguide

...

Training with:                                                                  
{'alpha': 0.7769943516287112, 'booster': 'gbtree', 'colsample_bylevel': 0.5939596761267271, 'colsample_bynode': 0.6753329089408893, 'colsample_bytree': 0.8580352445466448, 'eta': 0.48763282600959684, 'eval_metric': 'mae', 'gamma': 0.8639145515042577, 'grow_policy': 'depthwise', 'lambda': 1.311235915577443, 'max_delta_step': 1.87520984385713, 'max_depth': 5, 'min_child_weight': 0.45729596018009083, 'num_boost_round': 149, 'num_class': 100, 'objective': 'multi:softmax', 'sampling_method': 'uniform', 'subsample': 0.5513973549699895, 'tree_method': 'auto'}
Training with:                                                                  
{'alpha': 0.7224718999658483, 'booster': 'gbtree', 'colsample_bylevel': 0.7972744382347486, 'colsample_bynode': 0.5910589600815667, 'colsample_bytree': 0.9175520061065143, 'eta': 0.18833541792002378, 'eval_metric': 'mphe', 'gamma': 0.516708985584427, 'grow_policy': 'depthwise', 

Training with:                                                                  
{'alpha': 0.8665397618587577, 'booster': 'gbtree', 'colsample_bylevel': 0.8787136879124703, 'colsample_bynode': 0.5871003460372657, 'colsample_bytree': 0.9247488524988797, 'eta': 0.5363501845530937, 'eval_metric': 'mape', 'gamma': 1.7256336241877428, 'grow_policy': 'depthwise', 'lambda': 1.236806642235753, 'max_delta_step': 0.923951671479787, 'max_depth': 4, 'min_child_weight': 0.5481506145156494, 'num_boost_round': 127, 'num_class': 100, 'objective': 'multi:softmax', 'sampling_method': 'uniform', 'subsample': 0.7854777071072577, 'tree_method': 'auto'}
Training with:                                                                  
{'alpha': 0.8107470954602851, 'booster': 'gbtree', 'colsample_bylevel': 0.7833337678541561, 'colsample_bynode': 0.5620602672673933, 'colsample_bytree': 0.9563317976630605, 'eta': 0.3278762008933341, 'eval_metric': 'map', 'gamma': 0.5532444293254956, 'grow_policy': 'depthwise', '

Training with:                                                                  
{'alpha': 0.5841168850896621, 'booster': 'gbtree', 'colsample_bylevel': 0.772456275060236, 'colsample_bynode': 0.7380848508982734, 'colsample_bytree': 0.6348318220751411, 'eta': 0.6590333913512377, 'eval_metric': 'mphe', 'gamma': 0.38515804743101256, 'grow_policy': 'lossguide', 'lambda': 1.9068700375592222, 'max_delta_step': 0.8158990260523312, 'max_depth': 6, 'min_child_weight': 0.8893622753387774, 'num_boost_round': 189, 'num_class': 100, 'objective': 'multi:softmax', 'sampling_method': 'uniform', 'subsample': 0.7102090476469836, 'tree_method': 'exact'}
Training with:                                                                  
{'alpha': 0.722456330873367, 'booster': 'gbtree', 'colsample_bylevel': 0.5789151475884071, 'colsample_bynode': 0.6753726024334157, 'colsample_bytree': 0.6727419835159998, 'eta': 0.6273082308168069, 'eval_metric': 'mphe', 'gamma': 0.5273770655758787, 'grow_policy': 'depthwise'

Training with:                                                                  
{'alpha': 0.4953138959605138, 'booster': 'gbtree', 'colsample_bylevel': 0.830701567708358, 'colsample_bynode': 0.7931306604569377, 'colsample_bytree': 0.6435165767185452, 'eta': 0.6495964354382285, 'eval_metric': 'rmsle', 'gamma': 0.18167475941979816, 'grow_policy': 'lossguide', 'lambda': 0.5687036828719956, 'max_delta_step': 0.27656894853856095, 'max_depth': 1, 'min_child_weight': 1.2532159096971276, 'num_boost_round': 112, 'num_class': 100, 'objective': 'multi:softmax', 'sampling_method': 'uniform', 'subsample': 0.8659949604329873, 'tree_method': 'auto'}
Training with:                                                                  
{'alpha': 0.3540592132607944, 'booster': 'gbtree', 'colsample_bylevel': 0.8911219674912133, 'colsample_bynode': 0.8344095118827011, 'colsample_bytree': 0.5293727629754678, 'eta': 0.6652479589368148, 'eval_metric': 'rmsle', 'gamma': 0.18244856104224788, 'grow_policy': 'lossgu

Training with:                                                                  
{'alpha': 0.3293444422704934, 'booster': 'gbtree', 'colsample_bylevel': 0.820034599062519, 'colsample_bynode': 0.7567124430928299, 'colsample_bytree': 0.6650943876599144, 'eta': 0.9500308507514555, 'eval_metric': 'rmsle', 'gamma': 0.16919462298328308, 'grow_policy': 'lossguide', 'lambda': 0.4550395516806572, 'max_delta_step': 0.4931482899018738, 'max_depth': 4, 'min_child_weight': 2.477545038877734, 'num_boost_round': 100, 'num_class': 100, 'objective': 'multi:softmax', 'sampling_method': 'uniform', 'subsample': 0.8499808777199412, 'tree_method': 'auto'}
Training with:                                                                  
{'alpha': 0.30651932324585424, 'booster': 'gbtree', 'colsample_bylevel': 0.8973737619599349, 'colsample_bynode': 0.9242789325307459, 'colsample_bytree': 0.6861714057036433, 'eta': 0.8414286051694196, 'eval_metric': 'rmsle', 'gamma': 0.1327951226311447, 'grow_policy': 'lossguid

In [29]:
import tensorflow as tf
import tensorflow.keras as keras

from keras.models import Sequential
from keras.losses import SparseCategoricalCrossentropy
from keras.layers import Dense, Dropout

model = Sequential([
    Dense(1024),
    Dense(512),
    Dense(256),
    Dense(128),
    Dense(100)
])

model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

2024-01-24 14:06:55.169391: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-24 14:07:05.389910: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
model.fit(X_train, y_train, epochs=1000)

In [32]:
predictions = model.predict(X_test)
predictions = tf.nn.softmax(predictions)

score = 0
for i, pred in enumerate(predictions):
    final_pred = np.argmax(pred, 0)
    
    if final_pred == y_test[i]:
        score += 1

print(score/len(y_test))

32/32 [==============================] - 0s 1ms/step
0.583
